In [20]:
%run pair_classifier.py

In [49]:
import itertools
import os
import ntpath


def get_pairs(pair_names, splits):
    return [Pair(pair, splits) for pair in pair_names]

def get_playerid(name):
    return ntpath.basename(name)[:17]

def is_same_player(id1, id2):
    return 1 if id1 == id2 else 0

def get_ys(pairs):
    return [
        is_same_player(get_playerid(file0), get_playerid(file1))
        for file0,file1 in pairs
    ]


In [22]:
import random

def sample_filter(sample, prob):
    if sample[1] == 0:
        if random.random() < prob:
            return False
        else:
            return True
    else:
        return False

def sample(X, y, prob):
    combine = [(X[i], y[i]) for i in range(len(X))]
    sample = [s for s in combine if not sample_filter(s, prob)]
    
    return map(list, zip(*sample))

def count_sample(x):
    negatives = x.count(0)
    positives = x.count(1)
    return "{} negative samples and {} positive samples".format(negatives, positives)

In [23]:
def ml(X, y, pc, cv):
    skf = StratifiedKFold(cv)

    for train_index, test_index in skf.split(X, y):
        X_train = [X[i] for i in train_index]
        X_test = [X[i] for i in test_index]
        y_train = [y[i] for i in train_index]
        y_test = [y[i] for i in test_index]

        pc.train(X_train, y_train)
        pc.test(X_test, y_test)

In [40]:
path = "/cs/scratch/sy35/dota-data/16/data/mouseaction"
#pairs_1 = get_pairs(path, 1)
#pairs_2 = get_pairs(path, 2)
#pairs_3 = get_pairs(path, 3)
pairs = get_pair_names(path)
ys = get_ys(pairs)

In [41]:
count_sample(ys)

'584 negative samples and 66 positive samples'

In [46]:
pair_names, y = sample(pairs, ys, 0.2)

In [47]:
count_sample(y)

'113 negative samples and 66 positive samples'

In [50]:
pairs1 = get_pairs(pair_names, 1)

In [56]:
pairs2 = get_pairs(pair_names, 2)

In [59]:
pairs3 = get_pairs(pair_names, 3)

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = PairClassifier(LogisticRegression(class_weight="balanced"), 
                    LogisticRegression(class_weight="balanced"), 
                    LogisticRegression(class_weight="balanced"), 
                    (2,))

rf = PairClassifier(RandomForestClassifier(class_weight="balanced"), 
                    RandomForestClassifier(class_weight="balanced"), 
                    RandomForestClassifier(class_weight="balanced"), 
                    (2,))

In [95]:
ml(pairs1, y, lr, 5)

Accuracy: 0.648648648649, Precision: 0.529411764706, Recall: 0.642857142857
Accuracy: 0.527777777778, Precision: 0.3, Recall: 0.230769230769
Accuracy: 0.361111111111, Precision: 0.25, Recall: 0.384615384615
Accuracy: 0.4, Precision: 0.3, Recall: 0.461538461538
Accuracy: 0.714285714286, Precision: 0.6, Recall: 0.692307692308


In [96]:
ml(pairs1, y, rf, 5)

Accuracy: 0.783783783784, Precision: 0.75, Recall: 0.642857142857
Accuracy: 0.805555555556, Precision: 0.8, Recall: 0.615384615385
Accuracy: 0.583333333333, Precision: 0.333333333333, Recall: 0.153846153846
Accuracy: 0.657142857143, Precision: 0.52380952381, Recall: 0.846153846154
Accuracy: 0.771428571429, Precision: 0.666666666667, Recall: 0.769230769231


In [97]:
ml(pairs2, y, lr, 5)

Accuracy: 0.648648648649, Precision: 0.52380952381, Recall: 0.785714285714
Accuracy: 0.555555555556, Precision: 0.363636363636, Recall: 0.307692307692
Accuracy: 0.388888888889, Precision: 0.153846153846, Recall: 0.153846153846
Accuracy: 0.4, Precision: 0.25, Recall: 0.307692307692
Accuracy: 0.657142857143, Precision: 0.533333333333, Recall: 0.615384615385


In [98]:
ml(pairs2, y, rf, 5)

Accuracy: 0.72972972973, Precision: 0.625, Recall: 0.714285714286
Accuracy: 0.722222222222, Precision: 0.636363636364, Recall: 0.538461538462
Accuracy: 0.611111111111, Precision: 0.444444444444, Recall: 0.307692307692
Accuracy: 0.571428571429, Precision: 0.458333333333, Recall: 0.846153846154
Accuracy: 0.8, Precision: 0.8, Recall: 0.615384615385


In [99]:
ml(pairs3, y, lr, 5)

Accuracy: 0.648648648649, Precision: 0.529411764706, Recall: 0.642857142857
Accuracy: 0.444444444444, Precision: 0.181818181818, Recall: 0.153846153846
Accuracy: 0.388888888889, Precision: 0.153846153846, Recall: 0.153846153846
Accuracy: 0.514285714286, Precision: 0.375, Recall: 0.461538461538
Accuracy: 0.571428571429, Precision: 0.454545454545, Recall: 0.769230769231


In [100]:
ml(pairs3, y, rf, 5)

Accuracy: 0.648648648649, Precision: 0.538461538462, Recall: 0.5
Accuracy: 0.638888888889, Precision: 0.5, Recall: 0.384615384615
Accuracy: 0.638888888889, Precision: 0.5, Recall: 0.307692307692
Accuracy: 0.657142857143, Precision: 0.52380952381, Recall: 0.846153846154
Accuracy: 0.742857142857, Precision: 0.7, Recall: 0.538461538462


In [ ]:
from sklearn.neural_network import MLPClassifier


def get_pair_dfs(self, pair, splits):
        attacks, moves, casts = [], [], []
        file0, file1 = pair
        
        dfs = get_df(file0, file1, splits)
        
        return dfs[0], dfs[1], dfs[2]

class Pair:
    def __init__(self, pair, splits):
        attack_df, move_df, cast_df = get_pair_dfs(pair, splits)
        
        self.attack_df = attack_df
        self.move_df = move_df
        self.cast_df = cast_df
        

class PairClassifier:
    
    def __init__(self, attack_model, move_model, cast_model, network_size):
        self.attack_model = attack_model
        self.move_model = move_model
        self.cast_model = cast_model
        
        self.network = MLPClassifier(solver="lbfgs", hidden_layer_sizes=network_size, random_state=42)
        
    def get_pairs_dfs(self, pairs, splits):
        attacks, moves, casts = [], [], []
        for pair in pairs:
            attacks.append(pair.attack_df)
            moves.append(pair.move_df)
            casts.append(pair.cast_df)
            
        return pd.concat(attacks), pd.concat(moves). pd.concat(casts)
        
    def train(self, pairs, y, splits):
        for model, train_df in self.get_pairs_dfs(pairs, splits):
            model.fit(train_df, y)
        self.fit_network(pairs, y)
        
    def fit_network(self, pairs, y):
        X = [self.get_all_probas(pair) for pair in pairs]
        self.network.fit(X, y)
        
    def get_proba(self, model, df):
        return model.predict_proba(df)
        
    def get_all_probas(self, pair):
        attack_proba = self.get_proba(self.attack_model, pair.attack_df)
        move_proba = self.get_proba(self.move_model, pair.move_df)
        cast_proba = self.get_proba(self.cast_model, pair.cast_df)
        
        return [attack_proba[1], move_proba[1], cast_proba[1]]
        
    def predict(self, pairs):
        probabilities = [self.get_all_probas(pair) for pair in pairs]
        return self.network.predict(probabilities)

In [ ]:
path = "/cs/scratch/sy35/dota-data/tmp"
pairs = get_pairs(path)
ys = get_ys(pairs)
splits = 3

In [ ]:
X, y = sample(pairs, ys, 0.5)

In [ ]:
pc = PairClassifier(LogisticRegression(), LogisticRegression(), LogisticRegression(), (3,))

In [ ]:
pc.train(X, y, 2)